In [1]:
%load_ext autoreload
%autoreload 2

In [52]:
#import api_wrapper as wrap
from pairs_selection.api_wrapper import alph_api_wrapper as alph 
from pairs_selection.api_wrapper import alph_settings
import pandas as pd 
import numpy as np
from datetime import date, timedelta, date, datetime
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [24]:
date_range = []
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2021, 1, 1)
end_date = date(2022, 10,10)
for single_date in daterange(start_date, end_date):
    date_range.append(single_date.strftime("%Y-%m-%d"))

In [25]:
a_s = alph_settings(apikey = "MD27QTTVAK3AJBUQ", site = "https://www.alphavantage.co/query?", date_range = [datetime(2021, 8, 9), datetime(2022, 8, 9)])

In [26]:
function = "TIME_SERIES_INTRADAY_EXTENDED"
ticker_lst = ["IBM", "HPQ"]
interval = "60min"
slice1 = [f"year1month{i}" for i in range(1, 12)] #first three months of intraday data
slice2 = [f"year2month{i}" for i in range(1, 12)]
outputsize = "compact"
test = alph.intraday_url(a_s, function, ticker_lst, interval, slice = slice1+slice2, outputsize = outputsize)

ibm_url = test["IBM"][0]
ibm_obj = alph.get_csv_data(ibm_url, pretty_print = False)

hp_url = test["HPQ"][0]
hp_obj = alph.get_csv_data(hp_url, pretty_print = False)

ibm_url = []
for url in range(11,16):
    ibm_obj = alph.get_csv_data(test['IBM'][url], pretty_print = False)
    ibm_url += ibm_obj
    
    
hp_url = []
for url in range(11,16):
    hp_obj = alph.get_csv_data(test['HPQ'][url], pretty_print = False)
    hp_url += hp_obj

In [36]:
ibm_df = pd.DataFrame(ibm_url[1:], columns = ['time', 'open', 'high', 'low', 'close', 'volume'])
hp_df = pd.DataFrame(hp_url[1:], columns = ['time', 'open', 'high', 'low', 'close', 'volume'])

In [37]:
ibm_df.set_index(inplace=True, keys=['time'])
hp_df.set_index(inplace=True, keys=['time'])

In [38]:
ibm_df.sort_index(inplace=True)
hp_df.sort_index(inplace=True)

In [41]:
df = ibm_df.merge(hp_df, how='inner',left_index=True, right_index=True)
df = df[df["close_x"].str.contains("close") == False]

df['spread'] = df['close_x'].astype('float64') - df['close_y'].astype('float64')
data = df[['spread', 'volume_x','volume_y']]
data['volume_x'] = data['volume_x'].astype('float64')
data['volume_y'] = data['volume_y'].astype('float64')

/var/folders/fj/q_02z7zn39sbz9f_wqgkn5g40000gn/T/ipykernel_95935/2740859643.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['volume_x'] = data['volume_x'].astype('float64')
/var/folders/fj/q_02z7zn39sbz9f_wqgkn5g40000gn/T/ipykernel_95935/2740859643.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['volume_y'] = data['volume_y'].astype('float64')


In [42]:
data.dropna(inplace = True)

/var/folders/fj/q_02z7zn39sbz9f_wqgkn5g40000gn/T/ipykernel_95935/339833404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace = True)


In [44]:
def gen_labels(df,t,threshold):
    # Calculate % return on spread t hours later
    df['forward_return'] = df['spread'].diff(periods=t)/df['spread']
    
    #If the return is more than x%, we should have bought, and hence the label is (1)
    #If return is less than x%, we should have sold, and hence label is (-1)
    #If in between, do nothing (0)
    df['output'] = np.select([df['forward_return'] > threshold ,df['forward_return'] < -threshold],[1,-1])
    return df

data = gen_labels(data,t=24,threshold=0.02)

/var/folders/fj/q_02z7zn39sbz9f_wqgkn5g40000gn/T/ipykernel_95935/151558037.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['forward_return'] = df['spread'].diff(periods=t)/df['spread']
/var/folders/fj/q_02z7zn39sbz9f_wqgkn5g40000gn/T/ipykernel_95935/151558037.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['output'] = np.select([df['forward_return'] > threshold ,df['forward_return'] < -threshold],[1,-1])


In [46]:
split = round(0.8*len(data))
train, test = data[:split],data[split:]

x_train = train[['spread','volume_x','volume_y']]
y_train = train[['output']]

x_test = test[['spread','volume_x','volume_y']]
y_test = test[['output']]

In [47]:

param_grid={'penalty':['l1','l2'], 'C':[1e-3, 1e-2, 1e-1, 1, 10]}

# Create a based model
lr = LogisticRegression()
# Instantiate the grid search model
model = GridSearchCV(estimator=LogisticRegression(solver="liblinear"), param_grid=param_grid) #helps to test different kinds of hype params
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/uti

In [51]:
print(accuracy_score(y_pred, y_test))
print(model.best_params_)

0.5887445887445888
{'C': 0.01, 'penalty': 'l1'}


In [53]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       1.00      0.16      0.28        55
           0       0.57      1.00      0.73       127
           1       0.00      0.00      0.00        49

    accuracy                           0.59       231
   macro avg       0.52      0.39      0.34       231
weighted avg       0.55      0.59      0.47       231



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.